In [83]:
import pandas as pd
import os

幫合併完的資料增加日期的欄位

In [84]:
date =  pd.DataFrame()
date['日期'] = pd.DataFrame(pd.date_range(f'1/1/{2019}',f'31/12/{2022}'))
date['週'] = date['日期'].dt.isocalendar().week
date['年'] = date['日期'].dt.isocalendar().year
# date = date.drop(date[date['週'] == 53].index).reset_index()
# del_data =  date[date['週'] == 52].tail(1).index
# date = date.drop(date[del_data.values[0]+1:].index).reset_index()
# date["日期"] = pd.to_datetime(date["日期"] ,format='%Y/%m/%d')
date 


C:\Users\IDSL\AppData\Local\Temp\ipykernel_37240\4198979263.py:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  date['日期'] = pd.DataFrame(pd.date_range(f'1/1/{2019}',f'31/12/{2021}'))


,日期,週,年
0,2019-01-01,1,2019
1,2019-01-02,1,2019
2,2019-01-03,1,2019
3,2019-01-04,1,2019
4,2019-01-05,1,2019
...,...,...,...
1091,2021-12-27,52,2021
1092,2021-12-28,52,2021
1093,2021-12-29,52,2021
1094,2021-12-30,52,2021


讀取所有天氣資料檔案

In [85]:
total_data = pd.DataFrame()
for filename in os.listdir(f"./"):
    if filename=='天氣資料整理合併.ipynb' or filename=='.ipynb_checkpoints' or filename== '斗六降水量資料.csv':
        continue
    temp = pd.read_csv(f'./{filename}')
#     刪除第一筆
    temp = temp.drop([0])
    total_data = pd.concat([total_data, temp])
total_data = total_data.reset_index(drop=True)
# print(total_data)
total_data['日期'] = date['日期']
total_data['週'] = date['週']
total_data['年'] = date['年']
columns = ['日期','降水量(mm)','週','年','氣溫(℃)','相對溼度(%)']
# 只取日期和降水量
total_data = total_data[columns]
# 刪除第53週
# total_data = total_data.drop(total_data[total_data['週'] == 53].index).reset_index()
# total_data.to_csv(f'斗六降水量資料.csv',index=False, encoding='utf_8_sig') 
print(total_data)

             日期 降水量(mm)   週     年 氣溫(℃) 相對溼度(%)
0    2019-01-01     0.0   1  2019  17.5      87
1    2019-01-02     0.0   1  2019  18.1      90
2    2019-01-03     0.5   1  2019  19.0      95
3    2019-01-04     0.0   1  2019  20.2      92
4    2019-01-05     0.0   1  2019  21.5      87
...         ...     ...  ..   ...   ...     ...
1091 2021-12-27     1.0  52  2021  14.0      84
1092 2021-12-28     0.0  52  2021  16.6      81
1093 2021-12-29     0.0  52  2021  18.0      82
1094 2021-12-30     0.0  52  2021  16.8      79
1095 2021-12-31     0.0  52  2021  15.8      72

[1096 rows x 6 columns]


In [86]:
total_data['降水量(mm)'] = total_data['降水量(mm)'].apply(lambda x: float(x))
total_data['氣溫(℃)'] = total_data['氣溫(℃)'].apply(lambda x: float(x))
total_data['相對溼度(%)'] = total_data['相對溼度(%)'].apply(lambda x: int(x))

刪除53週並且整理一週幾天下雨幾天沒下雨'

In [94]:
# date = pd.read_csv(f'./斗六降水量資料.csv')
year = 2021
for num in range(1, 53,1):
    temp = total_data[(total_data['週'] == num)&(total_data['年'] == year)]
#         計算該周工作日
    rainy_day = len(temp[temp['降水量(mm)'] > 0])
    no_rainy_day = len(temp[temp['降水量(mm)'] == 0])
    tem_averange = temp['氣溫(℃)'].mean().round(2)
    humidity_averange = temp['相對溼度(%)'].mean().round(2)
   
    total_data.loc[(total_data['週'] == num)&(total_data['年'] == year),'下雨天數'] = rainy_day
    total_data.loc[(total_data['週'] == num)&(total_data['年'] == year),'沒下雨天數'] = no_rainy_day
    total_data.loc[(total_data['週'] == num)&(total_data['年'] == year),'平均氣溫'] = tem_averange
    total_data.loc[(total_data['週'] == num)&(total_data['年'] == year),'平均濕度'] = humidity_averange
   
#     if year == 2021 and num == 52:   
#         date.loc[date['週']==num,'休假天數'] = 2

# #     同一週期工作日、休假日都一樣所以只保留一筆
# date.drop_duplicates(subset='週', keep='last', inplace=True)
# date = date.drop(['日期', '休假日','level_0','index'], axis=1)
# print(date)
# answer = pd.merge(data, date, on='週')

In [96]:
temp = total_data[total_data['年'] == 2021]
temp

,日期,降水量(mm),週,年,氣溫(℃),相對溼度(%),下雨天數,沒下雨天數,平均氣溫,平均濕度
734,2021-01-04,0.0,1,2021,16.8,88,1.0,6.0,14.13,76.57
735,2021-01-05,0.0,1,2021,17.6,83,1.0,6.0,14.13,76.57
736,2021-01-06,0.0,1,2021,16.2,84,1.0,6.0,14.13,76.57
737,2021-01-07,2.5,1,2021,13.8,88,1.0,6.0,14.13,76.57
738,2021-01-08,0.0,1,2021,9.1,77,1.0,6.0,14.13,76.57
...,...,...,...,...,...,...,...,...,...,...
1091,2021-12-27,1.0,52,2021,14.0,84,1.0,4.0,16.24,79.60
1092,2021-12-28,0.0,52,2021,16.6,81,1.0,4.0,16.24,79.60
1093,2021-12-29,0.0,52,2021,18.0,82,1.0,4.0,16.24,79.60
1094,2021-12-30,0.0,52,2021,16.8,79,1.0,4.0,16.24,79.60


計算一週總降水量

In [101]:
# date = pd.read_csv(f'./斗六降水量資料.csv')
year = 2021
for num in range(1, 53,1):
    temp = total_data[(total_data['週'] == num)&(total_data['年'] == year)]
#         計算該周工作日
    total_rainy =  temp['降水量(mm)'].sum()
    total_data.loc[(total_data['週'] == num)&(total_data['年'] == year),'總下雨量'] = total_rainy
print(date)

             日期   週     年
0    2019-01-01   1  2019
1    2019-01-02   1  2019
2    2019-01-03   1  2019
3    2019-01-04   1  2019
4    2019-01-05   1  2019
...         ...  ..   ...
1091 2021-12-27  52  2021
1092 2021-12-28  52  2021
1093 2021-12-29  52  2021
1094 2021-12-30  52  2021
1095 2021-12-31  52  2021

[1096 rows x 3 columns]


In [102]:
total_data

,日期,降水量(mm),週,年,氣溫(℃),相對溼度(%),下雨天數,沒下雨天數,平均氣溫,平均濕度,總下雨量
0,2019-01-01,0.0,1,2019,17.5,87,1.0,5.0,19.33,90.0,0.5
1,2019-01-02,0.0,1,2019,18.1,90,1.0,5.0,19.33,90.0,0.5
2,2019-01-03,0.5,1,2019,19.0,95,1.0,5.0,19.33,90.0,0.5
3,2019-01-04,0.0,1,2019,20.2,92,1.0,5.0,19.33,90.0,0.5
4,2019-01-05,0.0,1,2019,21.5,87,1.0,5.0,19.33,90.0,0.5
...,...,...,...,...,...,...,...,...,...,...,...
1091,2021-12-27,1.0,52,2021,14.0,84,1.0,4.0,16.24,79.6,1.0
1092,2021-12-28,0.0,52,2021,16.6,81,1.0,4.0,16.24,79.6,1.0
1093,2021-12-29,0.0,52,2021,18.0,82,1.0,4.0,16.24,79.6,1.0
1094,2021-12-30,0.0,52,2021,16.8,79,1.0,4.0,16.24,79.6,1.0


In [37]:
# date.to_csv(f'斗六降水量資料.csv',index=False, encoding='utf_8_sig') 

In [103]:
total_data.to_csv(f'斗六降水量資料1.csv',index=False, encoding='utf_8_sig') 